# Demo Wine avec MLflow

## Setup MLflow
Par defaut suppose que le repository est le répertoire local mlruns

Pour travailler en coopération, on va désigner le serveur 
soit en ligne de commande avec
```
export MLFLOW_TRACKING_URI=http://localhost:5000
```
soit par programme

In [101]:
import mlflow
server_uri = "http://localhost:5000"
mlflow.set_tracking_uri(server_uri)

## Création d'une expérimentation

L'expérimentation peut être crée puis sélectionnée en ligne de commande par
```
mlflow experiments create --experiment-name ac2  --artifact-location /opt/mlflow/mlruns/
export MLFLOW_EXPERIMENT_ID=1
```
ou par programme (voir ci-après)

In [112]:
mlflow.set_experiment("wine5")

INFO: 'wine5' does not exist. Creating a new experiment


## Entraînement

In [73]:
import warnings
import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

import numpy as np
warnings.filterwarnings("ignore")
np.random.seed(40)

import mlflow.sklearn

In [74]:
import pandas as pd 

# read data from file
df = pd.read_csv("wine-quality.csv")

In [75]:
from sklearn.model_selection import train_test_split
# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(df)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

# make it easy to pass datasets
datasets = { 'train_x': train_x, 'train_y': train_y, 'test_x': test_x, 'test_y': test_y }

In [76]:
shapes = [ "%s : %s" % (name, dataset.shape) for (name,dataset) in datasets.items() ]
print(shapes)

['train_x : (3673, 11)', 'train_y : (3673, 1)', 'test_x : (1225, 11)', 'test_y : (1225, 1)']


In [46]:
def eval_parameters(in_alpha, in_l1_ratio):
    # Set default values if no alpha is provided
    alpha = float(in_alpha) if not in_alpha is None else 0.5
    # Set default values if no l1_ratio is provided
    l1_ratio = float(in_l1_ratio) if not in_l1_ratio is None else 0.5
    return alpha, l1_ratio

In [47]:
def eval_metrics(actual, predicted):
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    mae = mean_absolute_error(actual, predicted)
    r2 = r2_score(actual, predicted)
    return rmse, mae, r2


In [205]:
from sklearn.linear_model import lasso_path, enet_path
import matplotlib.pyplot as plt
from itertools import cycle
def plot_enet_descent_path(tmpdir, X, y, l1_ratio):
    # Compute paths
    eps = 5e-3  # the smaller it is the longer is the path

    # Reference the global image variable
    global image
    
    print("Computing regularization path using ElasticNet.")
    alphas_enet, coefs_enet, _ = enet_path(X, y, eps=eps, l1_ratio=l1_ratio, fit_intercept=False)

    # Display results
    fig = plt.figure(1)
    ax = plt.gca()

    colors = cycle(['b', 'r', 'g', 'c', 'k'])
    neg_log_alphas_enet = -np.log10(alphas_enet)
    for coef_e, c in zip(coefs_enet, colors):
        l1 = plt.plot(neg_log_alphas_enet, coef_e, linestyle='--', c=c)

    plt.xlabel('-Log(alpha)')
    plt.ylabel('coefficients')
    title = 'ElasticNet Path by alpha for l1_ratio = ' + str(l1_ratio)
    plt.title(title)
    plt.axis('tight')

    # Display images
    image = fig
    
    # Save figure
    fig.savefig(os.path.join(tempdir, "ElasticNet-paths.png"))

    # Close plot
    plt.close(fig)

    # Return images
    return image    


In [210]:
import tempfile
import shutil
import os
def train_elasticnet(in_alpha, in_l1_ratio, trial=None):
    from sklearn.linear_model import ElasticNet

    alpha, l1_ratio = eval_parameters(in_alpha, in_l1_ratio)
    print("Parameters (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    
    run_name = "en_%f_%f" % (alpha,l1_ratio)
    with mlflow.start_run() as run:
        # train with ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("r2", r2)
        mlflow.set_tag("algo", "ElastiNet")
        if trial != None:
             mlflow.set_tag("trial", trial)
        
        #run_id = run.info.run_id
        # store info       
        workdir = tempfile.mkdtemp()
        with tempfile.TemporaryDirectory() as tmpdirname:  
            coef_file_name = os.path.join(tmpdirname, "coefs.txt")
            with open(coef_file_name, "w") as f: 
                f.write("Coefs:\n")         
                print("\nCoefs:")
                [ f.write("\t %s: %s\n" % (name, coef)) for (name, coef) in zip(train_x.columns, lr.coef_) ]
                [ print("\t %s: %s" % (name, coef)) for (name, coef) in zip(train_x.columns, lr.coef_) ]
                f.write("\t intercept: %s\n" % lr.intercept_) 
                print("\t intercept: %s" % lr.intercept_) 

            # Call plot_enet_descent_path
            #image = plot_enet_descent_path(tmpdirname, train_x, train_y, l1_ratio)

            # Log artifacts (output files)
            mlflow.log_artifacts(tmpdirname, artifact_path="artifacts")


        # store model
        mlflow.sklearn.log_model(lr, "model")
        
        return rmse

In [208]:
train_elasticnet(0.5, 0.5) # alpha 0.5, L1 0.5

Parameters (alpha=0.500000, l1_ratio=0.500000):
Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160693
  R2: 0.12678721972772689

Coefs:
	 fixed acidity: -0.0
	 volatile acidity: -0.0
	 citric acid: 0.0
	 residual sugar: -0.0
	 chlorides: -0.0
	 free sulfur dioxide: 0.008224722161159152
	 total sulfur dioxide: -0.004286873122114217
	 density: -0.0
	 pH: 0.0
	 sulphates: 0.0
	 alcohol: 0.09612081747951785
	 intercept: [5.16789184]


0.82224284975954

In [211]:
train_elasticnet(0.5, 0.4)

Parameters (alpha=0.500000, l1_ratio=0.400000):
Elasticnet model (alpha=0.500000, l1_ratio=0.400000):
  RMSE: 0.8134651568415164
  MAE: 0.6249674191990106
  R2: 0.1453313314484228

Coefs:
	 fixed acidity: -0.0
	 volatile acidity: -0.0
	 citric acid: 0.0
	 residual sugar: -0.0
	 chlorides: -0.0
	 free sulfur dioxide: 0.008518403022712659
	 total sulfur dioxide: -0.003996324972212648
	 density: -0.0
	 pH: 0.0
	 sulphates: 0.0
	 alcohol: 0.1258556349366895
	 intercept: [4.80454277]


0.8134651568415164

In [ ]:
!pip install optuna

In [144]:
import optuna
from datetime import datetime
optuna.logging.set_verbosity(optuna.logging.INFO)
ts = datetime.now().strftime('%Y%m%dT%H%M%S')
# Define an objective function to be minimized.
def objective(trial):

    suggested_alpha = trial.suggest_uniform('alpha', 0.1, 0.8)
    suggested_l1_ratio = trial.suggest_uniform('l1_ratio', 0.1, 0.8)
        
    error = train_elasticnet(suggested_alpha, suggested_l1_ratio, trial="%s_%s" % (ts,trial.trial_id))
    
    return error  # A objective value linked with the Trial object.

study = optuna.create_study()  # Create a new study.
study.optimize(objective, n_trials=100)  # Invoke optimization of the objective function.

[W 2019-11-19 20:52:35,660] The use of `Trial.trial_id` is deprecated. Please use `Trial.number` instead.


Parameters (alpha=0.747833, l1_ratio=0.704219):
Elasticnet model (alpha=0.747833, l1_ratio=0.704219):
  RMSE: 0.8594278650219525
  MAE: 0.648058081447884
  R2: 0.04602117824947993


[I 2019-11-19 20:52:36,001] Finished trial#0 resulted in value: 0.8594278650219525. Current best value is 0.8594278650219525 with parameters: {'alpha': 0.7478331033104612, 'l1_ratio': 0.704218605906453}.
[W 2019-11-19 20:52:36,006] The use of `Trial.trial_id` is deprecated. Please use `Trial.number` instead.


Parameters (alpha=0.230129, l1_ratio=0.138279):
Elasticnet model (alpha=0.230129, l1_ratio=0.138279):
  RMSE: 0.7856004334637345
  MAE: 0.615250791861742
  R2: 0.20288074286893298


[I 2019-11-19 20:52:36,356] Finished trial#1 resulted in value: 0.7856004334637345. Current best value is 0.7856004334637345 with parameters: {'alpha': 0.23012921321484012, 'l1_ratio': 0.1382792394893255}.
[W 2019-11-19 20:52:36,358] The use of `Trial.trial_id` is deprecated. Please use `Trial.number` instead.


...

[I 2019-11-19 20:53:12,107] Finished trial#97 resulted in value: 0.788150958815487. Current best value is 0.7794260526252847 with parameters: {'alpha': 0.1027822033072079, 'l1_ratio': 0.10010821186339591}.
[W 2019-11-19 20:53:12,130] The use of `Trial.trial_id` is deprecated. Please use `Trial.number` instead.


Parameters (alpha=0.340320, l1_ratio=0.114356):
Elasticnet model (alpha=0.340320, l1_ratio=0.114356):
  RMSE: 0.7892284725918028
  MAE: 0.6168051880982486
  R2: 0.1955012721773578


[I 2019-11-19 20:53:12,498] Finished trial#98 resulted in value: 0.7892284725918028. Current best value is 0.7794260526252847 with parameters: {'alpha': 0.1027822033072079, 'l1_ratio': 0.10010821186339591}.
[W 2019-11-19 20:53:12,517] The use of `Trial.trial_id` is deprecated. Please use `Trial.number` instead.


Parameters (alpha=0.173062, l1_ratio=0.216709):
Elasticnet model (alpha=0.173062, l1_ratio=0.216709):
  RMSE: 0.7849805858551631
  MAE: 0.6151026353147879
  R2: 0.20413811881216049


[I 2019-11-19 20:53:12,887] Finished trial#99 resulted in value: 0.7849805858551631. Current best value is 0.7794260526252847 with parameters: {'alpha': 0.1027822033072079, 'l1_ratio': 0.10010821186339591}.


In [145]:
study.best_params

{'alpha': 0.1027822033072079, 'l1_ratio': 0.10010821186339591}

In [146]:
study.best_trial

FrozenTrial(number=43, state=TrialState.COMPLETE, value=0.7794260526252847, datetime_start=datetime.datetime(2019, 11, 19, 20, 52, 51, 221698), datetime_complete=datetime.datetime(2019, 11, 19, 20, 52, 51, 590577), params={'alpha': 0.1027822033072079, 'l1_ratio': 0.10010821186339591}, distributions={'alpha': UniformDistribution(high=0.8, low=0.1), 'l1_ratio': UniformDistribution(high=0.8, low=0.1)}, user_attrs={}, system_attrs={'_number': 43}, intermediate_values={}, trial_id=43)

In [147]:
trial_id = study.best_trial.number
print("trial_id: %s" % trial_id)
tag = "%s_%s" % (ts, trial_id)
print("tag: %s" % tag)

trial_id: 43
tag: 20191119T205235_43


In [152]:
from mlflow.entities import ViewType
query = "tags.trial = '%s'" % tag
runs = mlflow.search_runs(filter_string=query, run_view_type=ViewType.ACTIVE_ONLY)
runs.head(10)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.rmse,metrics.mae,metrics.r2,params.alpha,params.l1_ratio,tags.trial,tags.mlflow.source.name,tags.algo,tags.mlflow.user,tags.mlflow.source.type
0,cfe1faaad16b41d2bdb00b16f1a2d037,7,FINISHED,/opt/mlflow/mlruns/7/cfe1faaad16b41d2bdb00b16f...,2019-11-19 20:52:51.237000+00:00,2019-11-19 20:52:51.499000+00:00,0.779426,0.611384,0.215361,0.1027822033072079,0.10010821186339591,20191119T205235_43,/opt/miniconda/envs/demo/lib/python3.7/site-pa...,ElastiNet,ubuntu,LOCAL


In [159]:
best_model_id = runs['run_id'][0]
best_model_uri = runs['artifact_uri'][0]
print("best model - id: %s - uri: %s" % (best_model_id, best_model_uri) )

best model - id: cfe1faaad16b41d2bdb00b16f1a2d037 - uri: /opt/mlflow/mlruns/7/cfe1faaad16b41d2bdb00b16f1a2d037/artifacts


In [162]:
from  mlflow.tracking import MlflowClient
mlflow_client = MlflowClient()
mlflow_client.set_tag(best_model_id, "best_model", "true")

In [168]:
runs = mlflow.search_runs(filter_string=query, run_view_type=ViewType.ACTIVE_ONLY)
runs.head(10)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.rmse,metrics.mae,metrics.r2,params.alpha,params.l1_ratio,tags.trial,tags.mlflow.source.name,tags.algo,tags.mlflow.user,tags.mlflow.source.type,tags.best_model
0,cfe1faaad16b41d2bdb00b16f1a2d037,7,FINISHED,/opt/mlflow/mlruns/7/cfe1faaad16b41d2bdb00b16f...,2019-11-19 20:52:51.237000+00:00,2019-11-19 20:52:51.499000+00:00,0.779426,0.611384,0.215361,0.1027822033072079,0.10010821186339591,20191119T205235_43,/opt/miniconda/envs/demo/lib/python3.7/site-pa...,ElastiNet,ubuntu,LOCAL,true
